# Data filtering process
- 敬語タスク評価用テストセット[keicot_base.tsv](./data/keicot_base.tsv)の元となるデータセットは、KeiCOコーパスから常体・敬体文のペアを70対収集し、それに対して手動でアノテーションを行うことで作成されている
- アノテーションを行う前のデータフィルタリングプロセスを、参考までに以下に示す

## 1. KeiCOコーパスのダウンロード

In [2]:
# Note that the following line downloads the only version of KeiCO corpus even though the file name contains "OldVersion".
!wget -P data "https://raw.githubusercontent.com/Liumx2020/KeiCO-corpus/main/keico_corpus(forLREC)-OldVersion.csv"

--2023-08-30 23:07:17--  https://raw.githubusercontent.com/Liumx2020/KeiCO-corpus/main/keico_corpus(forLREC)-OldVersion.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 832609 (813K) [text/plain]
Saving to: ‘data/keico_corpus(forLREC)-OldVersion.csv’

keico_corpus(forLRE 100%[===================>] 813.09K  --.-KB/s    in 0.01s   

2023-08-30 23:07:17 (77.9 MB/s) - ‘data/keico_corpus(forLREC)-OldVersion.csv’ saved [832609/832609]



## 2. 敬体文を集める
### 2.1 敬語レベル2のデータに絞る
- 敬語レベル1,2が動詞敬語を含み、3は丁寧語、4は常体文（いわゆるタメ語）である
- 敬語レベル1はフィルタアウト
    - 文語調で日常会話とはかけ離れたもの (お出かけになられる)

In [22]:
import pandas as pd
keico_df = pd.read_csv('./data/keico_corpus(forLREC)-OldVersion.csv', header=0, names=['sentence','level','sh','oh','pl','field'], encoding='utf-8')
keico_df[keico_df.level == 1].head(10)

,sentence,level,sh,oh,pl,field
0,御病気なさったそうですね。回復されたのでしょうか？,1,1,0,0,挨拶
1,本日は温かい気候ですね。,1,1,0,0,挨拶
2,行ってらっしゃいませ。お気を付けて。,1,1,0,0,挨拶
3,皆様、御揃いでお出かけになられるのですか？,1,1,0,0,挨拶
4,本日はお早いのですね、お散歩ですか？,1,1,0,0,挨拶
5,お早うございます。本日も良いお天気ですね。,1,1,0,0,挨拶
6,お早うございます。早朝から御精が出ますね。,1,1,0,0,挨拶
7,こんにちは、お暑うございます。本日は蒸しますね。,1,1,0,0,挨拶
8,こんばんは、お寒うございます。随分ひえますね。,1,1,0,0,挨拶
9,生憎の御天気ですね。はっきりとしない生憎の御天気ですね。,1,1,0,0,挨拶


In [23]:
keico_df[keico_df.level == 2].head(10)

,sentence,level,sh,oh,pl,field
280,どうぞお許しください。,2,1,0,0,謝る
281,こちらの間違いでご迷惑をおかけし、大変申し訳ございません。,2,1,0,0,謝る
282,このたびは失礼をおかけして、大変申し訳ございません。,2,1,0,0,謝る
283,落選は、私の力不足によるものでした。,2,1,0,0,謝る
284,私に免じてお許しください。,2,1,0,0,謝る
285,私が至らなくてこんな事態になって申し訳ないです。,2,1,0,0,謝る
286,お部屋の家具を全部新しくされたんですね。,2,1,0,0,改める
287,その方針はやめられたほうがいいと思います。,2,1,0,0,改める
288,息子さん帰省されたのですね今朝港でお見掛けしました。,2,1,0,0,登場
289,先生がご挨拶にいらっしゃいました。,2,1,0,0,登場


In [25]:
keico_df_honorifics = keico_df[keico_df.level == 2]
len(keico_df_honorifics)

2046

### 2.2 フィールドを絞る
- ルールに基づいて活用をする動詞敬語を用いて分析したいので、定型表現の多そうな分野（「挨拶」など）は避ける
- できるだけ一般的なシチュエーションに絞る
- 謙譲語データを含みたい

In [26]:
keico_df_honorifics.field.unique()

array(['謝る', '改める', '登場', '歩く', '安心', '言う', '家', '怒る', '客', '挨拶', 'あいづち',
       'アイデア', '会う', '赤ちゃん', '遊ぶ', '季節', '集まる', '応募', '遠慮', '選ぶ', '受付',
       '祝う', '関係', '関心', '管理', '金', '返す', '帰る', '書く', '確認', '買う', '学校',
       'がっかり', '勝つ', 'お参り', 'お礼', '終わり', '獲得', 'メール', '送る', '教える', '驚く',
       '聞く', '嫌う', '着る', '暮らし向き', '計画', '公', '困る', 'いる', '探す', '作業', '心',
       '言葉', '断る', '作品', '裁く', '自覚', '叱る', '仕事', '知っている', '質問', '死ぬ',
       '支配', '準備', '招待', '食', '計算', '契約', '研究', '検討する', '体', '電話', '取り込む',
       '逃げる', '願う', '態度', '助ける', '楽しむ', '体格', '体験', '体調', '席', '相談', '贈答',
       '見る', '任せる', '待つ', '店', '別れる', '服を仕立てる', '謙る', '報告', '訪問', '褒める',
       '頼む', '忠告', '付き合う', '伝える', '調べる', '信じる', '心配', '好き', '反論', 'ひま',
       '秘密', '病状', '勧める', 'スポーツ', 'する', 'ねぎらう', '恥ずかしい', 'わかる', '行く',
       '政治的講演', '手紙', 'お知らせ', '急ぐ'], dtype=object)

In [27]:
df_filtered = keico_df_honorifics[
    (keico_df_honorifics.field == '遊ぶ') \
    | (keico_df_honorifics.field == '集まる') \
    | (keico_df_honorifics.field == '聞く') \
    | (keico_df_honorifics.field == '選ぶ') \
    | (keico_df_honorifics.field == '伝える') \
    | (keico_df_honorifics.field == '返す') \
    | (keico_df_honorifics.field == '帰る') \
    | (keico_df_honorifics.field == '頼む') \
    | (keico_df_honorifics.field == '驚く') \
    | (keico_df_honorifics.field == '心配') \
    | (keico_df_honorifics.field == '褒める') \
]
print(df_filtered.shape)
print(df_filtered[df_filtered.sh == 1].shape)
print(df_filtered[df_filtered.oh == 1].shape)

(124, 6)
(102, 6)
(22, 6)


## 3. 常体文を集める
- KeiCOコーパスは対訳コーパスではないが、「敬語レベルが異なるが元の文は同じ」データが多い
- 敬体文の収集と同じフィールド条件でフィルタリングし、敬体文とアラインすることで対訳データ対を作成する
    - 敬語レベル4（常体文）は167件あったので、対応する敬体文が見つからない43件をドロップした

In [29]:
keico_df_regular = keico_df[keico_df.level == 4][
    (keico_df.field == '遊ぶ') \
    | (keico_df.field == '集まる') \
    | (keico_df.field == '聞く') \
    | (keico_df.field == '選ぶ') \
    | (keico_df.field == '伝える') \
    | (keico_df.field == '返す') \
    | (keico_df.field == '帰る') \
    | (keico_df.field == '頼む') \
    | (keico_df.field == '驚く') \
    | (keico_df.field == '心配') \
    | (keico_df.field == '褒める') \
]
keico_df_regular.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167 entries, 4859 to 10004
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  167 non-null    object
 1   level     167 non-null    int64 
 2   sh        167 non-null    int64 
 3   oh        167 non-null    int64 
 4   pl        167 non-null    int64 
 5   field     167 non-null    object
dtypes: int64(4), object(2)
memory usage: 9.1+ KB


/tmp/ipykernel_468621/3805043312.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  keico_df_regular = keico_df[keico_df.level == 4][


## 4. 手動での修正
- 最終的に、124件のデータに対して追加の目視によるフィルタリングや修正を行い、70件のデータ対を得た
- 除外基準
    1. 名詞敬語（「お相子」）
    2. 状況や立場の条件を問わず使用される定型表現 (「よろしくお願いいたします」)
    3. 文語すぎる (「午前様」)
    4. 敬語が含まれていない
    5. 二者間ではない
- 修正基準
    1. 文語体　（「いらっしやいます」）
    2. 一貫性が低い （「さぞ驚かれたことと思います。驚かれるのは無理もないわ。」）